## Obtaining numerical solutions

This section presents some code that shows an approximation of what is happening inside the model we just built and ran.
This is a sanity check that the outputs look as we would expect if we had coded one of our simple models manually,
and is intended to provide some insight into what is going on under the surface.
Clearly, coding models in this explicit way would
make for overly verbose code if we tried to create a complicated, highly stratified model
in this way, and would not utilise many of the convenient features of the _summer_ platform.

In the example code below we use the Euler method to solve a system of ordinary differential equations (ODEs),
which is defined by the model's flows.
However, we don't typically use Euler in our modelling for policy, 
and a range of different solvers are available in the _summer_ backend.

In [ ]:
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi2
except:
  IN_COLAB = False

In [ ]:
import numpy as np
import pandas as pd
from typing import Dict

from summer2 import CompartmentalModel
from summer2.parameters import Parameter as param

pd.options.plotting.backend = "plotly"

## Declare parameters
First, we'll declare a set of parameters we can use with either of our two approaches.

In [ ]:
parameters = {
    "contact_rate": 1.,
    "recovery_rate": 0.333,
    "death_rate": 0.05,
    "population": 1000.,
    "seed": 10.,
    "start_time": 0.,
    "end_time": 20.,
    "time_step": 0.1,
}

## Explicit evaluation
The block comments in the following cell work through how this explicit approach to model evaluation can be performed.

In [ ]:
# Get the evaluation times based on the requested parameters
time_period = parameters["end_time"] - parameters["start_time"]
num_steps = int(time_period / parameters["time_step"]) + 1
times = np.linspace(parameters["start_time"], parameters["end_time"], num=num_steps)

# Prepare for outputs and populate the initial conditions
explicit_calcs = np.zeros((num_steps, 3))
explicit_calcs[0] = [
    parameters["population"] - parameters["seed"], 
    parameters["seed"],
    0.,
]

# Run the calculations at each modelled time step
for t_idx, t in enumerate(times):
    if t_idx == 0:
        continue

    # Get some quantities that we'll need later
    flow_rates = np.zeros(3)
    compartment_sizes = explicit_calcs[t_idx - 1]
    n_suscept = compartment_sizes[0]
    n_infect = compartment_sizes[1]
    n_pop = compartment_sizes.sum()
    
    # Apply the infection process (under the assumption of frequency-dependent transmission)
    force_of_infection = parameters["contact_rate"] * n_infect / n_pop
    infection_flow_rate = force_of_infection * n_suscept
    flow_rates[0] -= infection_flow_rate
    flow_rates[1] += infection_flow_rate

    # Recovery of the infectious compartment
    recovery_flow_rate = parameters["recovery_rate"] * n_infect
    flow_rates[1] -= recovery_flow_rate
    flow_rates[2] += recovery_flow_rate
    
    # Add an infection-specific death flow to the infectious compartment
    death_flow_rate = n_infect * parameters["death_rate"]
    flow_rates[1] -= death_flow_rate
    
    # Calculate compartment sizes at the next time step given the calculated flow rates
    explicit_calcs[t_idx] = compartment_sizes + flow_rates * parameters["time_step"]  
    
compartments = (
    "susceptible",
    "infectious",
    "recovered",
)
explicit_outputs_df = pd.DataFrame(explicit_calcs, columns=compartments, index=times)

## Comparison to the _summer_ approach
Now let's create the same system using a _summer_ compartmental model object.

In [ ]:
def get_sir_model(
    parameters: Dict,
) -> CompartmentalModel:
    """
    This is the same model as from notebook 01.
    
    Args:
        parameters: The parameter values to be used in running the model
    Returns:
        The summer model object
    """
    
    # Define the 
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    infectious_compartment = [
        "infectious",
    ]
    analysis_times = (
        parameters["start_time"], 
        parameters["end_time"]
    )

    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
        timestep=parameters["time_step"],
    )
    
    # Check and assign infectious seed
    pop = parameters["population"]
    seed = parameters["seed"]
    suscept_pop = pop - seed
    msg = "Seed larger than population"
    assert pop >= 0., msg
    
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    
    # Add a frequency-dependent transmission flow
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=param("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    
    # Add a constant recovery flow
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=param("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    
    # Add a constant infection-related death rate
    model.add_death_flow(
        name="infection_death", 
        death_rate=param("death_rate"), 
        source="infectious",
    )
    return model

## Plotting the to approaches
Last, let's confirm that the outputs are indeed the same with the two approaches.

In [ ]:
explicit_outputs_df.plot()

In [ ]:
sir_model = get_sir_model(parameters)
sir_model.run(parameters=parameters)
compartment_values = sir_model.get_outputs_df()
compartment_values.plot()